<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4: Nutri-Grade labels

---
## Problem Statement
Singaporeans are living longer but spending more time in ill-health. There are top 3 chronic medical conditions that Singaporeans suffer from are: Hypertension, Diabetes and Hyperlipidemia<br>

There are 3 main ways to prevent chronic illness:
- Physical Activity (Engage in at least 150-300 minutes of moderate-intensity aerobic activity in a week)
- Diet (Consume the receommended dietary allowances for sugar, saturated fat and salt)
- Healthy life choices (Avoid tobacco and excessive drinking)<br>

We will focus on the diet portion. More than half of Singaporeans’ daily sugar intake comes from beverages. This is why the government has came up with a nutri-grade labelling system in hopes that Singaporeans will reduce their sugar intake by making heatheir choices when choosing which drink to buy. However the nutrigrade labels only take into account trans fat and sugar and do not provide a holistic picture of the health of the drinks. Is there a way to create a more comprehensive indicator of how healthy drinks are?

## Contents:
- [Import libraries](#Import-libraries)
- [Import data](#Import-data) 
- [Merge datasets](#Merge-datasets)
- [Removal of duplicates](#Removal-of-duplicates)
- [Imput missing values](#Imput-missing-values)
- [Checking Duplicate Rows with Different Names](#Checking-Duplicate-Rows-with-Different-Names)
- [Creating columns for quantity, volume and serving size](#Creating-columns-for-quantity,-volume-and-serving-size)
- [Sugar Content](#Sugar-Content)
- [Protein Content](#Protein-Content)
- [Calorie/Energy Content](#Calorie/Energy-Content)
- [Fibre](#Fibre)
- [Carbohydrates](#Carbohydrates)
- [Cholesterol](#Cholesterol)
- [Sodium](#Sodium)
- [Fat and Saturated Fat](#Fat-and-Saturated-Fat)
- [Combination of Different Cleaned Dataframes](#Combination-of-Different-Cleaned-Dataframes)

# Import libraries

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import re

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Import data

In [2]:
drinks_df = pd.read_csv("../data/drinks.csv")
tea_df = pd.read_csv("../data/tea.csv")
juice_df = pd.read_csv("../data/juice.csv")
kopi_df = pd.read_csv("../data/kopi.csv")

# Merge datasets

In [3]:
# Merge the four csv into one file
merged_df = pd.concat([drinks_df, tea_df, juice_df, kopi_df], ignore_index=True)

In [4]:
# Checking if it is merged sucessfully 
merged_df

,Drink Volume,Drink Name,Attributes,Energy,Trans Fat,Protein,Total Fat,Saturated Fat,Cholesterol,Carbohydrate,...,Fibres,Total Calorieso,Calories From Saturated Fat,Salt (Sodium),Serving,TotalSugers,Monosaturated Fat,Satutated Fat,Total Dietary Fiber,Enery
0,24 x 200ml (CTN),Milo Chocolate Malt Milk UHT Packet Drink,Per Serving,1.7g,1mg,1.7g,0.9g,0g,9.5g,8g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12 x 320ml (CTN),Coca-Cola Can Drink - Zero Sugar,Per Serving (100ml),0kcal,NaN,0g,0g,NaN,NaN,0g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6 x 180ml,Coca-Cola Mini Can Drink - Zero Sugar,Per Serving (180ml),0kcal,NaN,0g,0g,NaN,NaN,0g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24 x 325ml (CTN),100 Plus Isotonic Can Drink - Original,Per Serving (100ml),27kcal,NaN,0g,0g,NaN,NaN,6.8g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24 x 200ml (CTN),Ribena Blackcurrant Fruit Packet Drink - Regular,Per Serving (100g),43kcal,0g,0g,0g,0g,NaN,10.6g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,250 G,Cafe specialists Traditional Signature Ground ...,Per Serving (2.5g),50kcal,NaN,2.5g,NaN,NaN,NaN,10g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
483,270 G,Mycofe Long Black O,Per Serving (18),NaN,NaN,0.9g,0g,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,6 X 260G,UCC Blended Coffee Luxurious Cafe Au Lait,Per Serving (),NaN,NaN,0.6g,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,6 X 185G,UCC Black 100% Roasted Coffee Sugar Free,Per Serving (),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Saving the merged datasets
merged_df.to_csv("../data/merged_uncleaned.csv", index=False)

# Removal of duplicates

After inspecting the datasets, there are duplicates in the merged datasets. We will remove the duplicates. The duplicates are there because certain drinks scraped from the NTUC website categorizes certain drinks such as tea & coffee. When scraping multiple types of drinks, we got duplicates of drinks. 

Converting text to lowercase ensures consistent, case-insensitive comparisons and processing, reducing variability and potential errors in textual data analysis therefore we would be converting everything to lowercase in the dataset.

In [6]:
# Converting to lowercase
merged_df = merged_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
merged_df.columns = merged_df.columns.str.lower()

In [7]:
# Checking for duplicates
duplicated_drink_names_count = merged_df['drink name'].duplicated().sum()
duplicated_drink_names_count

63

In [8]:
# Removing duplicates from drink names
merged_df.drop_duplicates(subset='drink name', keep='first', inplace=True)

In [9]:
# Saving cleaned file into merged_cleaned.csv
merged_df.to_csv("../data/merged_cleaned.csv", index=False)

In [10]:
# Reading the cleaned merged_cleaned dataset
cleaned_drink_df = pd.read_csv(r"../data/merged_cleaned.csv")

In [11]:
# Checking if duplicates has been sucessfully removed
cleaned_drink_df

,drink volume,drink name,attributes,energy,trans fat,protein,total fat,saturated fat,cholesterol,carbohydrate,...,fibres,total calorieso,calories from saturated fat,salt (sodium),serving,totalsugers,monosaturated fat,satutated fat,total dietary fiber,enery
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,1.7g,1mg,1.7g,0.9g,0g,9.5g,8g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),0kcal,NaN,0g,0g,NaN,NaN,0g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6 x 180ml,coca-cola mini can drink - zero sugar,per serving (180ml),0kcal,NaN,0g,0g,NaN,NaN,0g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24 x 325ml (ctn),100 plus isotonic can drink - original,per serving (100ml),27kcal,NaN,0g,0g,NaN,NaN,6.8g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24 x 200ml (ctn),ribena blackcurrant fruit packet drink - regular,per serving (100g),43kcal,0g,0g,0g,0g,NaN,10.6g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,250 g,cafe specialists traditional signature ground ...,per serving (2.5g),50kcal,NaN,2.5g,NaN,NaN,NaN,10g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,270 g,mycofe long black o,per serving (18),NaN,NaN,0.9g,0g,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,6 x 260g,ucc blended coffee luxurious cafe au lait,per serving (),NaN,NaN,0.6g,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,6 x 185g,ucc black 100% roasted coffee sugar free,per serving (),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There were 487 rows initially and after removing 63 duplicates, we are left with 424 rows.

# Imput missing values

There are also a lot of null values in the dataset. This is because not every ingredients/attributes are the same for each drinks. Therefore for null values, we will impute them with 0. 

In [12]:
# Imputing null values with 0
cleaned_drink_df.fillna(0, inplace=True)

In [13]:
# Check if null values has indeed been imputed with 0
cleaned_drink_df

,drink volume,drink name,attributes,energy,trans fat,protein,total fat,saturated fat,cholesterol,carbohydrate,...,fibres,total calorieso,calories from saturated fat,salt (sodium),serving,totalsugers,monosaturated fat,satutated fat,total dietary fiber,enery
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,1.7g,1mg,1.7g,0.9g,0g,9.5g,8g,...,0,0,0,0,0,0,0,0,0,0
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),0kcal,0,0g,0g,0,0,0g,...,0,0,0,0,0,0,0,0,0,0
2,6 x 180ml,coca-cola mini can drink - zero sugar,per serving (180ml),0kcal,0,0g,0g,0,0,0g,...,0,0,0,0,0,0,0,0,0,0
3,24 x 325ml (ctn),100 plus isotonic can drink - original,per serving (100ml),27kcal,0,0g,0g,0,0,6.8g,...,0,0,0,0,0,0,0,0,0,0
4,24 x 200ml (ctn),ribena blackcurrant fruit packet drink - regular,per serving (100g),43kcal,0g,0g,0g,0g,0,10.6g,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,250 g,cafe specialists traditional signature ground ...,per serving (2.5g),50kcal,0,2.5g,0,0,0,10g,...,0,0,0,0,0,0,0,0,0,0
420,270 g,mycofe long black o,per serving (18),0,0,0.9g,0g,0,0,0,...,0,0,0,0,0,0,0,0,0,0
421,6 x 260g,ucc blended coffee luxurious cafe au lait,per serving (),0,0,0.6g,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
422,6 x 185g,ucc black 100% roasted coffee sugar free,per serving (),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Saving the imputed data into another file. (Doing this because it gives us the option to revert to the unimputed dataset)
# The File used for modelling will be 'cleaned_drink_data.csv'
cleaned_drink_df.to_csv("../data/cleaned_drink_data.csv", index=False)

# Checking Duplicate Rows with Different Names

Some rows may contain different calculations with similar ingredients:
* Sugar
* Protein
* Calories
* Carbohydrates
* Fats

Data Cleaning will look at the basic micronutritional values that are emphasised in nutritional guidelines.

In [15]:
all_df = pd.read_csv("../data/cleaned_drink_data.csv")
all_df.head(2)

,drink volume,drink name,attributes,energy,trans fat,protein,total fat,saturated fat,cholesterol,carbohydrate,...,fibres,total calorieso,calories from saturated fat,salt (sodium),serving,totalsugers,monosaturated fat,satutated fat,total dietary fiber,enery
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,1.7g,1mg,1.7g,0.9g,0g,9.5g,8g,...,0,0,0,0,0,0,0,0,0,0
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),0kcal,0,0g,0g,0,0,0g,...,0,0,0,0,0,0,0,0,0,0


# Creating columns for quantity, volume and serving size

Many of the items are found in batch quantities e.g 12 x 320 mL, 24 x 200mL. This can be useful to distinguish between duplicate beverages overall.

The declaration of "serving size" and "servings per package" in the Nutrition Informattion Packaging (NIP) are only required when the nutrients present are declared on a per serving basis.

Most food products in the market are labelled with NIPs, although this information is only compulsory when a nutrition or health claim on food is made.

A nutrition claim suggests or implies that a food has a nutritive property. Examples include, claims like "High in fibre", "Low in fat", "Cholesterol free", "Sugar free". Nutrition claim is a voluntary statement which describes the amount of nutrient in a food or a group of foods.

Citations:
https://www.sfa.gov.sg/food-information/labelling-packaging-information/understanding-food-nutrition-labels 

In [16]:
# Find all relevant columns
columns_of_interest = ["drink volume", "drink name", "attributes", "serving", "per serving", "per serving size", "serving per",
                       "serving size: 4.5 tbsp"]
sizes = all_df.loc[:, columns_of_interest]
sizes

,drink volume,drink name,attributes,serving,per serving,per serving size,serving per,serving size: 4.5 tbsp
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,0,0,0,0,0
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),0,0,0,0,0
2,6 x 180ml,coca-cola mini can drink - zero sugar,per serving (180ml),0,0,0,0,0
3,24 x 325ml (ctn),100 plus isotonic can drink - original,per serving (100ml),0,0,0,0,0
4,24 x 200ml (ctn),ribena blackcurrant fruit packet drink - regular,per serving (100g),0,0,0,0,0
...,...,...,...,...,...,...,...,...
419,250 g,cafe specialists traditional signature ground ...,per serving (2.5g),0,0,0,0,0
420,270 g,mycofe long black o,per serving (18),0,0,0,0,0
421,6 x 260g,ucc blended coffee luxurious cafe au lait,per serving (),0,0,0,0,0
422,6 x 185g,ucc black 100% roasted coffee sugar free,per serving (),0,0,0,0,0


In [17]:
#spit drink volume column by x. If have x, will have 2 elements. If not, 1 or 0 elements.
sizes['quantity'] = ""
sizes['volume'] = ""

for index, row in sizes.iterrows():
    values = str(row["drink volume"])
    values = values.split('x')
    
    
    if len(values) == 2:
        sizes.at[index, 'quantity'] = values[0]
        sizes.at[index, 'volume'] = values[1]
    elif len(values) == 1:
        sizes.at[index, 'quantity'] = 1
        sizes.at[index, 'volume'] = values[0]
    else:
        sizes.at[index, 'quantity'] = 1
        sizes.at[index, 'volume'] = 0

sizes.head(2)

,drink volume,drink name,attributes,serving,per serving,per serving size,serving per,serving size: 4.5 tbsp,quantity,volume
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,0,0,0,0,0,24,200ml (ctn)
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),0,0,0,0,0,12,320ml (ctn)


In [18]:
# Use regex to extract values within parentheses
sizes['Serving Size_Cleaned'] = sizes['attributes'].str.extract(r'\((.*?)\)').fillna("")
sizes.head(5)

,drink volume,drink name,attributes,serving,per serving,per serving size,serving per,serving size: 4.5 tbsp,quantity,volume,Serving Size_Cleaned
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,0,0,0,0,0,24,200ml (ctn),
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),0,0,0,0,0,12,320ml (ctn),100ml
2,6 x 180ml,coca-cola mini can drink - zero sugar,per serving (180ml),0,0,0,0,0,6,180ml,180ml
3,24 x 325ml (ctn),100 plus isotonic can drink - original,per serving (100ml),0,0,0,0,0,24,325ml (ctn),100ml
4,24 x 200ml (ctn),ribena blackcurrant fruit packet drink - regular,per serving (100g),0,0,0,0,0,24,200ml (ctn),100g


In [19]:
sizes[sizes['Serving Size_Cleaned'] == ""].head(20) #106 rows that still lack serving size

,drink volume,drink name,attributes,serving,per serving,per serving size,serving per,serving size: 4.5 tbsp,quantity,volume,Serving Size_Cleaned
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,0,0,0,0,0,24,200ml (ctn),
11,4 x 125g,milo chocolate malt uht packet drink - less sugar,per serving,0,0,0,0,0,4,125g,
20,1.5l,pokka bottle drink - japanese roasted tea houj...,per serving,0,0,0,0,0,1,1.5l,
44,500ml,milo chocolate malt milk bottle drink - iced e...,per serving,0,0,0,0,0,1,500ml,
62,6 x 240ml,pokka coffee can drink - milk coffee (less sugar),per serving,0,0,0,0,0,6,240ml,
91,4 x 200ml,fever-tree bottle - tonic water (mediterranean),per serving,0,0,0,0,0,4,200ml,
99,350g,bonchoco instant hot chocolate drink,per 100g,0,0,0,0,0,1,350g,
103,4 x 250ml,alchemy & tonic grapefruit & thyme tonic water,per serving (),0,0,0,0,0,4,250ml,
104,4 x 250ml,alchemy & tonic hibiscus pink lemonade,per serving (),0,100ml,0,0,0,4,250ml,
108,1 l,del monte four seasons juice drink tetra,per serving (),0,0,0,0,0,1,1 l,


In [20]:
# Inputting value of serving size if found in columns 3 to 8.
mask = sizes.index[sizes['Serving Size_Cleaned'] == ""].tolist()
servings_columns = sizes.columns[3:8]

sizes[servings_columns] = sizes[servings_columns].astype(str)
sizes.loc[mask, 'Serving Size_Cleaned'] = sizes.loc[mask, servings_columns].apply("".join, axis=1).str.lstrip('0').str.rstrip('0')
sizes.loc[mask,:].head(20)

,drink volume,drink name,attributes,serving,per serving,per serving size,serving per,serving size: 4.5 tbsp,quantity,volume,Serving Size_Cleaned
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,0,0,0,0,0,24,200ml (ctn),
11,4 x 125g,milo chocolate malt uht packet drink - less sugar,per serving,0,0,0,0,0,4,125g,
20,1.5l,pokka bottle drink - japanese roasted tea houj...,per serving,0,0,0,0,0,1,1.5l,
44,500ml,milo chocolate malt milk bottle drink - iced e...,per serving,0,0,0,0,0,1,500ml,
62,6 x 240ml,pokka coffee can drink - milk coffee (less sugar),per serving,0,0,0,0,0,6,240ml,
91,4 x 200ml,fever-tree bottle - tonic water (mediterranean),per serving,0,0,0,0,0,4,200ml,
99,350g,bonchoco instant hot chocolate drink,per 100g,0,0,0,0,0,1,350g,
103,4 x 250ml,alchemy & tonic grapefruit & thyme tonic water,per serving (),0,0,0,0,0,4,250ml,
104,4 x 250ml,alchemy & tonic hibiscus pink lemonade,per serving (),0,100ml,0,0,0,4,250ml,100ml
108,1 l,del monte four seasons juice drink tetra,per serving (),0,0,0,0,0,1,1 l,


In [21]:
sizes = sizes.drop(servings_columns, axis=1)
sizes

,drink volume,drink name,attributes,quantity,volume,Serving Size_Cleaned
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,24,200ml (ctn),
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),12,320ml (ctn),100ml
2,6 x 180ml,coca-cola mini can drink - zero sugar,per serving (180ml),6,180ml,180ml
3,24 x 325ml (ctn),100 plus isotonic can drink - original,per serving (100ml),24,325ml (ctn),100ml
4,24 x 200ml (ctn),ribena blackcurrant fruit packet drink - regular,per serving (100g),24,200ml (ctn),100g
...,...,...,...,...,...,...
419,250 g,cafe specialists traditional signature ground ...,per serving (2.5g),1,250 g,2.5g
420,270 g,mycofe long black o,per serving (18),1,270 g,18
421,6 x 260g,ucc blended coffee luxurious cafe au lait,per serving (),6,260g,
422,6 x 185g,ucc black 100% roasted coffee sugar free,per serving (),6,185g,


## Sugar Content

In [22]:
#Find all columns with Sugar. Sugar is measured in grams (g)
sugars = all_df.filter(regex='sug')
sugars.head(10)

,sugars,sugar,total sugar,total sugars,added sugars,added sugar,includes added sugars,sugars.1,of which sugars,totalsugers
0,0.5g,0,0,0,0,0,0,0,0,0
1,0g,0,0,0,0,0,0,0,0,0
2,0g,0,0,0,0,0,0,0,0,0
3,6.8g,0,0,0,0,0,0,0,0,0
4,0,10.5g,0,0,0,0,0,0,0,0
5,25.0g,0,0,0,0,0,0,0,0,0
6,0,0,22.0g,0,0,0,0,0,0,0
7,0g,0,0,0,0,0,0,0,0,0
8,0,0,0,12.2g,0,0,0,0,0,0
9,0g,0,0,0,0,0,0,0,0,0


In [23]:
#Find all columns with Sugar. Sugar is measured in grams (g)
for col in sugars.columns:
    sugars[col] = sugars[col].str.replace('g', '').fillna(0)
    sugars[col] = pd.to_numeric(sugars[col], errors='coerce') # 'coerce' turns non-numeric values to numeric

combined_sugars = ["sugars.1", "sugars", "sugar", "of which sugars"]
total_sugar_amounts = ["total sugars", "total sugar", "totalsugers"]
total_added_sugars = ["added sugars", "added sugar", "includes added sugars"]

# Sum the columns
sugars['Sugar_combined (g)'] = sugars[combined_sugars].sum(axis=1)
sugars['Total_Sugar_combined (g)'] = sugars[total_sugar_amounts].sum(axis=1)
sugars['Added_Sugar_combined (g)'] = sugars[total_added_sugars].sum(axis=1)

# Drop duplicate columns
sugars = sugars.drop(combined_sugars + total_sugar_amounts + total_added_sugars, axis=1)
sugars.head(10)

,Sugar_combined (g),Total_Sugar_combined (g),Added_Sugar_combined (g)
0,0.5,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,6.8,0.0,0.0
4,10.5,0.0,0.0
5,25.0,0.0,0.0
6,0.0,22.0,0.0
7,0.0,0.0,0.0
8,0.0,12.2,0.0
9,0.0,0.0,0.0


## Protein Content

In [24]:
#Find all columns with Protein. Protein is measured in grams (g)
pps = all_df.filter(regex='protein')
pps

,protein,proteins
0,1.7g,0
1,0g,0
2,0g,0
3,0g,0
4,0g,0
...,...,...
419,2.5g,0
420,0.9g,0
421,0.6g,0
422,0,0


In [25]:
# Remove grams and convert to numeric.
protein_columns = pps.columns
for col in protein_columns:
    pps[col] = pps[col].str.replace('g', '').fillna(0)
    pps[col] = pd.to_numeric(pps[col], errors='coerce') # 'coerce' turns non-numeric values to numeric

pps[pps["proteins"] > 0]

,protein,proteins
140,0.0,1.1
285,0.0,1.1


In [26]:
pps['protein_total (g)'] = pps[protein_columns].sum(axis=1)
protein = pps.drop(protein_columns, axis=1)
protein.iloc[[140,285],:]

,protein_total (g)
140,1.1
285,1.1


## Calorie/Energy Content

Calories, or Energy, can be measured in calories (cal) or kilocalories (kcal). It has also be measured in joules, kilojoules and even grams. Hence, it has to be standardised and cleaned.

In [27]:
#Find all columns with Calories. Calories or energy is same thing, can be measured in grams, kcal or joules
calories = all_df.filter(regex='([Cc]alorie|ener)')
calories.head(5)

,energy,calories,calories from fat,calorie,calories from fat.1,total energy,total calories,energy from fat,total calorieso,calories from saturated fat,enery
0,1.7g,0,0,0,0,0,0,0,0,0,0
1,0kcal,0,0,0,0,0,0,0,0,0,0
2,0kcal,0,0,0,0,0,0,0,0,0,0
3,27kcal,0,0,0,0,0,0,0,0,0,0
4,43kcal,0,0,0,0,0,0,0,0,0,0


In [28]:
#Find all columns with calories. Calories is measured in calories (kcal or cal) or kilojoules(1 Calorie equals 4.184 kJ)
# Function to convert calorie values to kcal
def cal_to_kcal(calorie_str):
    size = len(str(calorie_str))
    
    if isinstance(calorie_str, str):
        # If the input is already an integer, return it as is
        calorie_str = calorie_str.lower()
        
        # if value contains 2 measurements e.g kcal/kj
        if len(calorie_str.split("\\")) == 2 or len(calorie_str.split("/")) == 2:
            calorie_str = calorie_str.split('\\')[0]
            calorie_str = calorie_str.split('/')[0]
            size = len(str(calorie_str))
            
    
    if isinstance(calorie_str, int):
        # If the input is already an integer, return it as is
        return calorie_str
    elif 'kcal' in calorie_str:
        # Convert kcal to integer
        numeric_part = calorie_str[: size-4]
        return float(numeric_part)
    elif 'kcl' in calorie_str:
        # Convert kcal to integer
        numeric_part = calorie_str[: size-3]
        return float(numeric_part)
    elif 'cal' in calorie_str:
        # Convert cal to integer after dividing by 1000
        return int(float(calorie_str.replace('cal', '')) / 1000)
    elif 'kj' in calorie_str:
        numeric_part = calorie_str[: size-2]
        return float(numeric_part) // 4.184  # Convert from kJ to kcal
    elif 'j' in calorie_str:
        numeric_part = calorie_str[: size-1]
        return float(numeric_part) * 0.000239006   # Convert from J to kcal
    elif 'g' in calorie_str:
        numeric_part = calorie_str[: size-1]
        return float(numeric_part) * 0.009 # Convert from kJ to kcal
    else:
        # Convert '0' to integer
        return int(calorie_str)

# Apply the process_cell function to each cell in the DataFrame
calories = calories.applymap(cal_to_kcal)
calories.head(5)

,energy,calories,calories from fat,calorie,calories from fat.1,total energy,total calories,energy from fat,total calorieso,calories from saturated fat,enery
0,0.0153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,43.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
#Combine 2 Calories/Energy from Fat together
calories['Combined Calories from Fat (kcal)'] = calories.iloc[:,2] + calories.iloc[:,4] + calories.iloc[:,9] + calories.iloc[:,7]
calories["Calories (kcal)"] = calories["calories"] + calories["calorie"]
calories["Total Calories (kcal)"] = calories["total calories"] + calories["total calorieso"] 


calories = calories.drop(calories.columns[[2, 4, 7,9]],axis = 1)
calories = calories.drop(["calories", "calorie", "total calories", "total calorieso"], axis=1)

#calories = calories.drop(['Calories from Fat', "Calories From Fat"], axis=1)
calories.head(5)

,energy,total energy,enery,Combined Calories from Fat (kcal),Calories (kcal),Total Calories (kcal)
0,0.0153,0.0,0.0,0.0,0.0,0.0
1,0.0000,0.0,0.0,0.0,0.0,0.0
2,0.0000,0.0,0.0,0.0,0.0,0.0
3,27.0000,0.0,0.0,0.0,0.0,0.0
4,43.0000,0.0,0.0,0.0,0.0,0.0


Check if there are any rows with both energy and calories
- If there are no overlapping cells, they can be combined.

In [30]:
# Check if any rows have both total energy and total calories
calories[(calories['total energy'] > 0) & (calories['Total Calories (kcal)'] > 0)]

,energy,total energy,enery,Combined Calories from Fat (kcal),Calories (kcal),Total Calories (kcal)


In [31]:
calories[(calories['energy'] > 0) & (calories['Calories (kcal)'] > 0)]

,energy,total energy,enery,Combined Calories from Fat (kcal),Calories (kcal),Total Calories (kcal)
333,443.0,0.0,0.0,0.135,60.0,0.0


In [32]:
calories["Total Calories (kcal)"] = calories["total energy"] + calories["Total Calories (kcal)"] 
calories["Calories (kcal)"] = calories["Calories (kcal)"] + calories["energy"] + calories["enery"]
calories = calories.drop(["total energy", "energy", "enery"], axis=1)

# Nutrition Labels are listed by serving, manual correction of 1 value
calories.loc[333, "Total Calories (kcal)"] = 443
calories.loc[333, "Calories (kcal)"] = 60.0
calories.loc[333,:]

Combined Calories from Fat (kcal)      0.135
Calories (kcal)                       60.000
Total Calories (kcal)                443.000
Name: 333, dtype: float64

In [33]:
calories.head(20)

,Combined Calories from Fat (kcal),Calories (kcal),Total Calories (kcal)
0,0.0,0.0153,0.0
1,0.0,0.0000,0.0
2,0.0,0.0000,0.0
3,0.0,27.0000,0.0
4,0.0,43.0000,0.0
5,0.0,100.0000,0.0
6,0.0,88.0000,0.0
7,0.0,0.0000,0.0
8,0.0,96.0000,0.0
9,0.0,0.0000,0.0


# Fibre

Fibre is usually measured in terms of grams (g). The Singapore Health Promotion Board recommends a daily dietary fibre intake of 20g for women and 26g for men. This equates to 2 servings of fruit and 2 servings of vegetables per day.

Dietary fibre comprises any substance of the plant origin, which is not digestible by your body. It includes plant cell walls and supporting structures. Dietary fibre means carbohydrate polymers with ten or more monomeric units, which are not hydrolysed by the endogenous enzymes in the small intestine of humans. 

Fibre consists of 2 components: Insoluble and Soluble

Citation:
https://www.healthxchange.sg/food-nutrition/food-tips/are-you-eating-enough-fibre 
https://www.hpb.gov.sg/docs/default-source/pdf/hdp-nutrition-guidelines-(may-2020).pdf?sfvrsn=f72b9650_2 

In [34]:
ffs = all_df.filter(regex='(fibre|fiber)')
ffs.head(2)

,dietary fibre,fibre,fiber,dietary fibre.1,dietry fibre,dietary fiber,dietary fiber.1,fibres,total dietary fiber
0,30mg,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0


In [35]:
#Find all columns with calories. Calories is measured in calories (kcal or cal) or kilojoules(1 Calorie equals 4.184 kJ)
# Function to convert calorie values to kcal
def mg_to_g(fib_str):
    
    size = len(str(fib_str))
    
    if isinstance(fib_str, str):
        # If the input is already an integer, return it as is
        fib_str = fib_str.lower()
    
    if isinstance(fib_str, int):
        # If the input is already an integer, return it as is
        return fib_str
    elif 'mg' in fib_str:
        # Convert mg to integer
        numeric_part = fib_str[: size-2]
        return float(numeric_part) * 0.001
    elif '<' in fib_str:
        numeric_part = ''.join(filter(str.isdigit, fib_str))
        return float(numeric_part)
    elif 'g' in fib_str:
        # Convert g to integer
        numeric_part = fib_str[: size-1]
        return float(numeric_part)
    else:
        # Convert '0' to integer
        return int(fib_str)

# Apply the process_cell function to each cell in the DataFrame
ffs = ffs.applymap(mg_to_g)
ffs.head(20)

,dietary fibre,fibre,fiber,dietary fibre.1,dietry fibre,dietary fiber,dietary fiber.1,fibres,total dietary fiber
0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Check if there are overlapping labels of Dietary Fibre and Fibre

Although they refer to the same thing, there may be misclassifications present.

In [36]:
ffs[(ffs['dietary fibre'] > 0) & (ffs['fibre'] > 0)]

,dietary fibre,fibre,fiber,dietary fibre.1,dietry fibre,dietary fiber,dietary fiber.1,fibres,total dietary fiber


In [37]:
# Find rows with more than 1 non-zero value
# Based on this, means every row only has maximum 1 fiber value.
filtered_df = ffs[(ffs != 0).sum(axis=1) > 1]
filtered_df

,dietary fibre,fibre,fiber,dietary fibre.1,dietry fibre,dietary fiber,dietary fiber.1,fibres,total dietary fiber


In [38]:
# Check amount of total dietary fiber. Only 1 row. Based on content, it should be dietary fiber by serving.
ffs[ffs["total dietary fiber"] > 0]

,dietary fibre,fibre,fiber,dietary fibre.1,dietry fibre,dietary fiber,dietary fiber.1,fibres,total dietary fiber
416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6


In [39]:
# Sum all columns together
ffs_columns = ffs.columns
ffs['Fibre (g)'] = ffs.sum(axis=1)
fibre = ffs.drop(ffs_columns, axis=1)
fibre.tail(15)

,Fibre (g)
409,0.0
410,0.0
411,0.0
412,4.8
413,0.1
414,0.0
415,2.1
416,1.6
417,0.0
418,0.0


# Carbohydrates

Measured in grams

In [40]:
carbs = all_df.filter(regex='carb')
carbs.head(5)

,carbohydrate,total carbohydrate,carbohydrates,total carbohydrates,bicarbonates,total carboyhdrate,carbohyrate,total carb
0,8g,0,0,0,0,0,0,0
1,0g,0,0,0,0,0,0,0
2,0g,0,0,0,0,0,0,0
3,6.8g,0,0,0,0,0,0,0
4,10.6g,0,0,0,0,0,0,0


In [41]:
def carb_filter(carb_str, default_value="g"):
    
    size = len(str(carb_str))
    
    
    if isinstance(carb_str, str):
        # If the input is already an integer, return it as is
        carb_str = carb_str.lower()
        
        # Condition where nutrition value has written words of "less than"
        if "less than" in carb_str:
            carb_str = carb_str.replace('less than', '<')
        
        carb_str = carb_str.replace('<', '')
    
    # Conditonal statement to convert certain values with certain measurements based on gram, mg
    if isinstance(carb_str, int):
        # If the input is already an integer, return it as is
        return carb_str
    elif 'mg' in carb_str:
        # Convert mg to integer
        numeric_part = carb_str.replace('mg', '')
        value = float(numeric_part) * 0.001
        
        #if default_value is changed to mg
        if default_value == "mg":
            value = value * 1000
        return value
    
    
    
    elif 'g' in carb_str:
        # Convert g to integer
        numeric_part = carb_str.replace('g', '')
        value = float(numeric_part)
        
        #if default_value is changed to mg
        if default_value == "mg":
            value = value * 1000
        return value
    
    elif 'kcal' in carb_str:
        # Convert kcal to grams
        numeric_part = carb_str[: size-4]
        value = float(numeric_part) * 9000
        
        #if default_value is changed to mg
        if default_value == "mg":
            value = value * 1000
        
        return value
    
    elif 'cal' in carb_str:
        # Convert cal to grams
        numeric_part = carb_str[: size-4]
        value = float(numeric_part) * 9
        
        #if default_value is changed to mg
        if default_value == "mg":
            value = value * 1000
        
        return value
    
    elif 'j' in carb_str:
        # convert joule to grams
        numeric_part = carb_str[: size-1]
        return float(numeric_part) * 37700
        
        #if default_value is changed to mg
        if default_value == "mg":
            value = value * 1000
    
    else:
        # Convert '0' to integer
        return int(carb_str)
        
        
    
carbs = carbs.applymap(carb_filter)
carbs.head(5)

,carbohydrate,total carbohydrate,carbohydrates,total carbohydrates,bicarbonates,total carboyhdrate,carbohyrate,total carb
0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# To check if there are any values that possess more than 1 value
filtered_carbs = carbs[(carbs != 0).sum(axis=1) > 1]
filtered_carbs

,carbohydrate,total carbohydrate,carbohydrates,total carbohydrates,bicarbonates,total carboyhdrate,carbohyrate,total carb


In [43]:
carbs[carbs["total carb"] >0]

,carbohydrate,total carbohydrate,carbohydrates,total carbohydrates,bicarbonates,total carboyhdrate,carbohyrate,total carb
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0


In [44]:
carbcol = carbs.columns
carbs['Carbohydrates (g)'] = carbs.drop('bicarbonates', axis=1).sum(axis=1)
carbohydrates = carbs.drop(carbcol, axis=1)
carbohydrates

,Carbohydrates (g)
0,8.0
1,0.0
2,0.0
3,6.8
4,10.6
...,...
419,10.0
420,16.5
421,7.8
422,0.0


# Cholesterol

Usually measured in milligrams (mg). Cholesterol is a fat-like substance that is either produced by the body or derived from food. It is necessary for the body to function normally and is used to build cell membranes, however, only a small amount of cholesterol in the blood is required to meet the body’s needs. 

Lipids (fat) such as cholesterol are insoluble in water and therefore carried throughout the body by special proteins called lipoproteins. There are two types of lipoproteins, which are commonly known as low-density lipoprotein (LDL) and high-density lipoprotein (HDL).

Citations:
https://www.healthxchange.sg/high-cholesterol/essential-guide-high-cholesterol/good-bad-cholesterol-truth-hdl-ldl 

In [45]:
chol = all_df.filter(regex='chol')
chol.head(5)

,cholesterol,choline,cholestrol
0,9.5g,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [46]:
chol = chol.applymap(carb_filter, default_value = "mg")
chol["cholesterol (mg)"] = chol["cholesterol"] + chol["cholestrol"] + chol["choline"]
cholesterol = chol.drop(["choline", "cholestrol", "cholesterol"], axis=1)
cholesterol.head(5)

,cholesterol (mg)
0,9500.0
1,0.0
2,0.0
3,0.0
4,0.0


# Sodium

Sodium is usually measured in milligrams (mg). Salt consumption among Singaporeans has increased tremendously. Findings by the 2018/2019 National Nutrition Survey conducted by the Health Promotion Board revealed that 90 per cent of Singaporeans consumed 9g of salt daily on average, which is well above the recommended amount of 5g per day.

Citation:
https://www.singhealth.com.sg/news/singapore-health/salt-intake-rising-in-singapore 

In [47]:
ss = all_df.filter(regex='sodi|salt|natri')
ss

,sodium,natrium / sodium,salt,salt equivalent,natrium,salt (sodium)
0,60mg,0,0,0,0,0
1,8mg,0,0,0,0,0
2,11mg,0,0,0,0,0
3,48mg,0,0,0,0,0
4,0mg,0,0,0,0,0
...,...,...,...,...,...,...
419,50mg,0,0,0,0,0
420,0,0,17.4mg,0,0,0
421,0,0,0,0.02g,0,0
422,0,0,0,0,0,0


In [48]:
ss = ss.applymap(carb_filter, default_value = "mg")

In [49]:
# To check if there are any rows with more than 1 value
filtered_ss = ss[(ss != 0).sum(axis=1) > 1]
filtered_ss

,sodium,natrium / sodium,salt,salt equivalent,natrium,salt (sodium)


In [50]:
ss[(ss["natrium"] > 0)]

,sodium,natrium / sodium,salt,salt equivalent,natrium,salt (sodium)
344,0.0,0.0,0.0,0.0,800.0,0.0


In [51]:
sscol = ss.columns
ss["Sodium Content (mg)"] = ss.sum(axis=1)
sodium = ss.drop(sscol, axis = 1)
sodium

,Sodium Content (mg)
0,60.0
1,8.0
2,11.0
3,48.0
4,0.0
...,...
419,50.0
420,17.4
421,20.0
422,0.0


# Fat and Saturated Fat

Fats and Saturated Fats are usually measured in grams(g)

In [52]:
fats = all_df.filter(regex='fat|lipid|satura')
# remove columns that contain the word "calorie"
fats = fats[fats.columns.drop(list(fats.filter(regex='calorie|sulfate|energy')))]
fats.columns

Index(['trans fat', 'total fat', 'saturated fat', 'monounsaturated fat',
       'polyunsaturated fat', 'trans fatty acid', 'fat', 'fats', 'total fats',
       'saturated fat.1', 'trans fat.1', 'saturated fats', 'total fat.1',
       'saturated fats.1', 'saturates', 'fat.1', 'saturates.1', 'fats.1',
       'saturaed fat', 'monounsaturated fatty acid',
       'polyunsaturated fatty acid', 'saturated fatty acid', 'unsaturated fat',
       'saturated', 'lipids', 'saturates fat', 'saturatedfat',
       'of which saturated fats', 'monosaturated fat', 'satutated fat'],
      dtype='object')

In [53]:
fats.head(5)

,trans fat,total fat,saturated fat,monounsaturated fat,polyunsaturated fat,trans fatty acid,fat,fats,total fats,saturated fat.1,...,polyunsaturated fatty acid,saturated fatty acid,unsaturated fat,saturated,lipids,saturates fat,saturatedfat,of which saturated fats,monosaturated fat,satutated fat
0,1mg,0.9g,0g,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0g,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0g,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0g,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0g,0g,0g,0g,0g,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
fats = fats.applymap(carb_filter)
fats.head(5)

,trans fat,total fat,saturated fat,monounsaturated fat,polyunsaturated fat,trans fatty acid,fat,fats,total fats,saturated fat.1,...,polyunsaturated fatty acid,saturated fatty acid,unsaturated fat,saturated,lipids,saturates fat,saturatedfat,of which saturated fats,monosaturated fat,satutated fat
0,0.001,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### To check the breakdown and classification of fat labels 

There are different types of fat in our diet and each one of them has a different effect on health. Based on their composition, we can classify fats as saturated, polyunsaturated, monounsaturated and trans fats. As such, it is important to correctly identify these different types of fats in order to give a more accurate label.

Saturated fat and trans fat raise LDL cholesterol ("bad" cholesterol) and this increases the risk of developing heart disease and stroke. Saturated fat is found mostly in animal fat (e.g. fatty meat, chicken skin), full fat dairy products, deep fried food and food prepared with palm-based vegetable oils. Food containing trans fat include pastries, cakes, cookies and products made with vegetable shortening and hydrogenated or partially hydrogenated oils.

- **Saturated fats**, while not as harmful as trans fats, by comparison with unsaturated fats negatively impact health and are best consumed in moderation.
- **Trans fats** consumption will increase disease risk, even when eaten in small quantities.

Scientists generally agree that unsaturated fats (Monounsaturated and polyunsaturated fats) - olive, canola, soya, corn, and peanut oils - are healthier choices. Unsaturated fats, rich in omega 3 fats offer a multitude of heart-health benefits ranging from lowering triglycerides to improving the elasticity of blood vessels. But, all healthier oils are to be used sparingly as part of a well-balanced diet.



Citation:
* https://www.hpb.gov.sg/newsroom/article/eating-for-healthy-ageing 
* https://www.hsph.harvard.edu/nutritionsource/what-should-you-eat/fats-and-cholesterol/#:~:text=Saturated%20fats%2C%20while%20not%20as,%2C%20cheese%2C%20and%20ice%20cream

In [55]:
# To check if there are any rows with more than 1 value
filtered_fats = fats[(fats != 0).sum(axis=1) > 1]
filtered_fats

,trans fat,total fat,saturated fat,monounsaturated fat,polyunsaturated fat,trans fatty acid,fat,fats,total fats,saturated fat.1,...,polyunsaturated fatty acid,saturated fatty acid,unsaturated fat,saturated,lipids,saturates fat,saturatedfat,of which saturated fats,monosaturated fat,satutated fat
0,0.001,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.000,2.8,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.000,1.8,1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.000,2.8,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,0.000,0.0,0.0,0.0,0.0,0.0,1.6,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0
412,0.200,8.4,7.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0
415,0.000,0.5,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# To check if there are any values that express both serving 
fats[(fats['trans fat'] > 0) & (fats['trans fatty acid'] > 0)]

,trans fat,total fat,saturated fat,monounsaturated fat,polyunsaturated fat,trans fatty acid,fat,fats,total fats,saturated fat.1,...,polyunsaturated fatty acid,saturated fatty acid,unsaturated fat,saturated,lipids,saturates fat,saturatedfat,of which saturated fats,monosaturated fat,satutated fat


### Creating Consolidated Fat Columns

As there are many duplicates and various terminologies used, we will be consolidating these columns together to ensure more accurate and precise classification. In addition, preliminary feature engineering can be conducted to bolster categorising efforts moving ahead.

Consolidated categories for trans fat, saturated fats, unsaturated fats will be made.

In [57]:
#Creating trans fat column
t_fats = ["trans fat", "trans fatty acid", "trans fat.1"]
fats["trans_fat_combined (g)"] = fats[t_fats].sum(axis=1)

In [58]:
#Creating saturated fat column
s_fats = ["saturated fat", "saturated fat.1", "saturated fats", "saturated fats.1", "saturaed fat", "saturates fat",
         "saturatedfat",'of which saturated fats', 'satutated fat', "saturates", "saturates.1", "saturated fatty acid", 
         "saturated"]

fats["saturated_fat_combined (g)"] = fats[s_fats].sum(axis=1)

In [59]:
#Creating other fat column

# listing of relevant columns
mono_unfat = ["monounsaturated fat", "monounsaturated fatty acid", "monosaturated fat", "unsaturated fat"]
poly_unfat = ["polyunsaturated fat", "polyunsaturated fatty acid"]
total_fat = ["total fat", "total fats", "total fat.1"]
small_fat = ["fat", "fats", "fat.1", "fats.1", "lipids"]

# Creating new columns
fats["monounsaturated_fat_combined (g)"] = fats[mono_unfat].sum(axis=1)
fats["polyunsaturated_fat_combined (g)"] = fats[poly_unfat].sum(axis=1)
fats["total_fat_combined (g)"] = fats[total_fat].sum(axis=1)
fats["fat_combined (g)"] = fats[small_fat].sum(axis=1)

#dropping all redundant columns
all_columns_to_drop = t_fats + s_fats + mono_unfat + poly_unfat + total_fat + small_fat
fats = fats.drop(all_columns_to_drop, axis=1)
fats

,trans_fat_combined (g),saturated_fat_combined (g),monounsaturated_fat_combined (g),polyunsaturated_fat_combined (g),total_fat_combined (g),fat_combined (g)
0,0.001,0.0,0.0,0.0,0.9,0.0
1,0.000,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,0.0,0.0,0.0,0.0
3,0.000,0.0,0.0,0.0,0.0,0.0
4,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
419,0.000,0.0,0.0,0.0,0.0,0.0
420,0.000,0.0,0.0,0.0,0.0,0.0
421,0.000,0.0,0.0,0.0,0.0,0.7
422,0.000,0.0,0.0,0.0,0.0,0.0


# Combination of Different Cleaned Dataframes 

In [60]:
list_of_dfs = [sizes, sugars, protein, calories, fibre, carbohydrates,cholesterol, sodium, fats]

cleaned_df = pd.concat(list_of_dfs, axis=1)
cleaned_df

,drink volume,drink name,attributes,quantity,volume,Serving Size_Cleaned,Sugar_combined (g),Total_Sugar_combined (g),Added_Sugar_combined (g),protein_total (g),...,Fibre (g),Carbohydrates (g),cholesterol (mg),Sodium Content (mg),trans_fat_combined (g),saturated_fat_combined (g),monounsaturated_fat_combined (g),polyunsaturated_fat_combined (g),total_fat_combined (g),fat_combined (g)
0,24 x 200ml (ctn),milo chocolate malt milk uht packet drink,per serving,24,200ml (ctn),,0.5,0.0,0.0,1.7,...,0.03,8.0,9500.0,60.0,0.001,0.0,0.0,0.0,0.9,0.0
1,12 x 320ml (ctn),coca-cola can drink - zero sugar,per serving (100ml),12,320ml (ctn),100ml,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,8.0,0.000,0.0,0.0,0.0,0.0,0.0
2,6 x 180ml,coca-cola mini can drink - zero sugar,per serving (180ml),6,180ml,180ml,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,11.0,0.000,0.0,0.0,0.0,0.0,0.0
3,24 x 325ml (ctn),100 plus isotonic can drink - original,per serving (100ml),24,325ml (ctn),100ml,6.8,0.0,0.0,0.0,...,0.00,6.8,0.0,48.0,0.000,0.0,0.0,0.0,0.0,0.0
4,24 x 200ml (ctn),ribena blackcurrant fruit packet drink - regular,per serving (100g),24,200ml (ctn),100g,10.5,0.0,0.0,0.0,...,0.00,10.6,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,250 g,cafe specialists traditional signature ground ...,per serving (2.5g),1,250 g,2.5g,0.0,0.0,0.0,2.5,...,0.00,10.0,0.0,50.0,0.000,0.0,0.0,0.0,0.0,0.0
420,270 g,mycofe long black o,per serving (18),1,270 g,18,14.8,0.0,0.0,0.9,...,1.20,16.5,0.0,17.4,0.000,0.0,0.0,0.0,0.0,0.0
421,6 x 260g,ucc blended coffee luxurious cafe au lait,per serving (),6,260g,,0.0,0.0,0.0,0.6,...,0.00,7.8,0.0,20.0,0.000,0.0,0.0,0.0,0.0,0.7
422,6 x 185g,ucc black 100% roasted coffee sugar free,per serving (),6,185g,,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0


In [61]:
# Save to .csv file
cleaned_df.to_csv('../data/cleaned_dataset_without_extra_nutrients.csv', index=False)

1. We will now classify the drinks into 4 different health classes (A,B,C,D). A being the healthiest choice.
2. This classifier is according to the Health Promotion Board
3. https://hpb.gov.sg/healthy-living/food-beverage/nutri-grade <--- Source
4. We have added in another condition here which is sodium as it is not included in the nutri-grade labels.

In [62]:
# For sugar:
# Create a new column "Combined_Sugar (g)" with the sum of the two columns
cleaned_df["Combined_Sugar (g)"] = cleaned_df["Sugar_combined (g)"] + cleaned_df["Total_Sugar_combined (g)"]

# For fat:
# Create a new column "Combined_Fat (g)" with the sum of the two columns
cleaned_df["Combined_Fat (g)"] = cleaned_df["fat_combined (g)"] +cleaned_df["total_fat_combined (g)"]

# Drop the original columns for both sugar and fat
cleaned_df.drop(["Sugar_combined (g)", "Total_Sugar_combined (g)", "fat_combined (g)", "total_fat_combined (g)"], axis=1, inplace=True)


In [63]:
def classify_drink(row):
    # Classify based on Sugar Content
    sugar_content = row['Combined_Sugar (g)']
    if sugar_content < 1:
        sugar_class = 'A'
    elif sugar_content <= 5:
        sugar_class = 'B'
    elif sugar_content <= 10:
        sugar_class = 'C'
    else:
        sugar_class = 'D'

    # Classify based on Saturated Fat Content
    saturated_fat_content = row['Combined_Fat (g)']
    if saturated_fat_content < 0.7:
        fat_class = 'A'
    elif saturated_fat_content <= 1.2:
        fat_class = 'B'
    elif saturated_fat_content <= 2.8:
        fat_class = 'C'
    else:
        fat_class = 'D'

    # Classify based on Sodium Content
    # Convert sodium content from mg to g
    sodium_content = row['Sodium Content (mg)'] / 1000
    if sodium_content < 1:
        sodium_class = 'A'
    elif sodium_content <= 5:
        sodium_class = 'B'
    elif sodium_content <= 10:
        sodium_class = 'C'
    else:
        sodium_class = 'D'

    # Return the worst grade among the three nutrients
    return max(sugar_class, fat_class, sodium_class)

# Apply the classification function to the dataset
cleaned_df['nutrigrade'] = cleaned_df.apply(classify_drink, axis=1)

# Display the drink names along with their classification
classification_result = cleaned_df[['drink name', 'nutrigrade']]
classification_result.head()


,drink name,nutrigrade
0,milo chocolate malt milk uht packet drink,B
1,coca-cola can drink - zero sugar,A
2,coca-cola mini can drink - zero sugar,A
3,100 plus isotonic can drink - original,C
4,ribena blackcurrant fruit packet drink - regular,D


In [64]:
# Saving this file to be used for modelling.
cleaned_df.to_csv('../data/model_csv.csv', index=False)

This marks the end of cleaning the dataset.
